In [17]:
#-=-=-=-=-=-=-=-= GET DRAFT HIST =-=-=-=-=-=-=-=-#

##import libs##
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define years to pull 
## current year finalizes to /draftyear but you can specify it still
years = range(2000, 2059) #iterate over years +1 as it'll stop the year before python is retarded
base_url = "https://atl-01.statsplus.net/skylinebaseball/draftyear/?year="

# Init 
all_players = []

# Loop through each year
for year in years:
    url = f"{base_url}{year}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find the draft table
    table = soup.find("table", {"class": "drafttable"})
    if not table:
        print(f"Could not find table for year {year}")
        continue
    
    # Get all rows 
    rows = table.find("tbody").find_all("tr")
    
    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 10:  # Ensure the row has the required number of columns
            continue
        
        # Rip data
        rnd = cols[0].text.strip()
        pick = cols[1].text.strip()
        ovr = cols[2].text.strip()
        team = cols[3].text.strip()
        pos = cols[4].text.strip()
        
        # Extract player data and ID
        player_link = cols[5].find("a")
        player_name = player_link.text.strip() if player_link else None
        player_id = player_link['data-player-id'] if player_link and 'data-player-id' in player_link.attrs else None

        #gonna add these in even if summary stats are garb just bec it can be a good check for YoY if I can pull those and roll them up today.
        age = cols[6].text.strip()
        bat_war = cols[7].text.strip()
        pitch_war = cols[8].text.strip()
        total_war = cols[9].text.strip()
        
        # Append to all_players list
        all_players.append({
            "Year": year,
            "Rnd": rnd,
            "Pick": pick,
            "Ovr": ovr,
            "Team": team,
            "Pos": pos,
            "Player": player_name,
            "Player ID": player_id,
            "Age": age,
            "Bat WAR": bat_war,
            "Pitch WAR": pitch_war,
            "Total WAR": total_war
        })

# Create a DataFrame and save to CSV
df = pd.DataFrame(all_players)
print(df)
df.to_csv("draft_data.csv", index=False)

       Year Rnd Pick   Ovr Team Pos            Player Player ID Age Bat WAR  \
0      2000   1    2     2  NYM  CF    Chris Anderson     18137  79   22.73   
1      2000   1    3     3  COL   P     Mike Brickman     18758  79   -0.27   
2      2000   1    4     4   TB  SS      Jon Meredith     17941  76   -0.31   
3      2000   1    5     5  STL  CF  Amari Rockingham     18381  79   10.10   
4      2000   1    6     6  SEA   P    Jarrod Bendorf     18710  75   -0.22   
...     ...  ..  ...   ...  ...  ..               ...       ...  ..     ...   
63759  2058  35   32  1262  COL   P       Jalen White    121083  18    0.00   
63760  2058  35   33  1263  NYY   P       Ben McBride    121359  18    0.00   
63761  2058  35   34  1264  ATL  CF   Colton Morrison    121369  18    0.00   
63762  2058  35   35  1265  MIA   P      Anthony Delp    114881  22    0.00   
63763  2058  35   36  1266  TEX   P   Glenn Alexander    118826  21    0.00   

      Pitch WAR Total WAR  
0         -0.03     22.